In [63]:
import pandas as pd
import numpy as np
import pickle
from collections import defaultdict

In [35]:
pred_df = pd.read_pickle('pred.pkl')
pred_df

,pseudo_gen_seq,code_gen_seq,pred_code_gen_seq
0,"[set, [CPY], to, [CPY]]","[[CPY], =, [CPY], ;]","[[CPY], =, [CPY], ;]"
1,"[[CPY], [CPY], is, [CPY]]","[[CPY], (, [CPY], ==, [CPY], )]","[[CPY], (, [CPY], ==, [CPY], )]"
2,"[read, [CPY], and, [CPY]]","[cin, >>, [CPY], >>, [CPY], ;]","[cin, >>, [CPY], >>, [CPY], ;]"
3,"[declare, [CPY], longs, [CPY], [CPY], [CPY], a...","[[CPY], [CPY], [CPY], [CPY], [CPY], ,, [CPY], ...","[[CPY], [CPY], [CPY], [CPY], [CPY], ,, [CPY], ..."
4,"[[CPY], [CPY], integer, array, where, the, the...","[int, [CPY], [CPY], [CPY], [CPY], {, [CPY], [C...","[int, [CPY], [CPY], [CPY], [CPY], [CPY], [CPY]..."
...,...,...,...
181857,"[declare, [CPY], constant, integer, [CPY], [CP...","[[CPY], const, int, [CPY], [CPY], [CPY], ;]","[const, int, [CPY], [CPY], [CPY], ;]"
181858,"[print, [CPY], and, a, new, line]","[cout, <<, "", [CPY], "", <<, ', \n, ', ;]","[cout, <<, [CPY], <<, endl, ;]"
181859,"[change, the, value, of, [CPY], to, [CPY], [CP...","[[CPY], =, [CPY], [CPY], [CPY], [CPY], [CPY], ...","[[CPY], =, [CPY], [CPY], [CPY], [CPY], [CPY], ..."
181860,"[[CPY], [CPY], [CPY], is, less, than, [CPY]]","[[CPY], [CPY], (, [CPY], <, [CPY], )]","[[CPY], [CPY], (, [CPY], <, [CPY], )]"


## Accuracy scores

#### Strict accuracy

Exact sentence matches

In [3]:
strict_bools = pred_df.apply(lambda row: np.array_equal(row['code_gen_seq'], row['pred_code_gen_seq']), axis=1)
strict_accuracy = strict_bools.sum() / len(pred_df)
strict_accuracy

0.6979687895217253

In [4]:
def compare_cols(row, col1, col2):
    """
    Count of equal words in two rows 
    """
    try:
        comps = np.array(row['code_gen_seq']) == np.array(row['pred_code_gen_seq'])
    except ValueError as e:
        comps = np.array([False])
    return np.sum(comps)

In [54]:
word_matches = np.sum(pred_df.apply(compare_cols, args=('code_gen_seq', 'pred_code_gen_seq'), axis=1))
total_words = np.sum(pred_df.apply(lambda row: np.size(row['code_gen_seq']), axis=1))

word_accuracy = word_matches / total_words
word_accuracy

/Users/vibhamasti/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  


0.5843358298790285

## Combine

In [30]:
train_df = pd.read_csv('../../data/input-tok-train-shuf.tsv', sep='\t', header=None)
train_df.columns = ['pseudo', 'code']
train_df

,pseudo,code
0,set l to mid,l = mid ;
1,if i is 0,if ( i == 0 )
2,read n and k,cin >> n >> k ;
3,declare long longs sum = 0 and min = LONG_LONG...,"long long min = LONG_LONG_MAX , sum = 0 ;"
4,dy = integer array where the the following int...,"int dy [ ] = { 0 , 0 , - 1 , 1 } ;"
...,...,...
181857,declare static constant integer mod = 1000000009,static const int mod = 1000000009 ;
181858,print NO and a new line,"cout << "" NO "" << ' \n ' ;"
181859,change the value of ans to abs ( x - y ) / d,ans = abs ( x - y ) / d ;
181860,else if s is less than f,else if ( s < f )


In [93]:
cpy_dataset = pd.read_pickle('../../data/CPY_dataset.pkl')
cpy_dataset = cpy_dataset.drop(columns=['code_gen_seq', 'code_gen_seq_aug'])
cpy_dataset

,pseudo_gen_seq
0,"[set, [CPY], to, [CPY]]"
1,"[[CPY], [CPY], is, [CPY]]"
2,"[read, [CPY], and, [CPY]]"
3,"[declare, [CPY], longs, [CPY], [CPY], [CPY], a..."
4,"[[CPY], [CPY], integer, array, where, the, the..."
...,...
181857,"[declare, [CPY], constant, integer, [CPY], [CP..."
181858,"[print, [CPY], and, a, new, line]"
181859,"[change, the, value, of, [CPY], to, [CPY], [CP..."
181860,"[[CPY], [CPY], [CPY], is, less, than, [CPY]]"


In [33]:
import importlib
import preprocessor
importlib.reload(preprocessor)

<module 'preprocess' from '/Users/vibhamasti/Personal/College/Sem_6/NLP/Pseudocode_to_code/spoc/copy_generation/combine/preprocess.py'>

In [94]:
preprocessor.tokenize_column(train_df, col_to_tokenize='code', tokenized_col_name='code_tokens', inplace=True)
preprocessor.tokenize_column(train_df, col_to_tokenize='pseudo', tokenized_col_name='pseudo_tokens', inplace=True)
train_df.drop(columns=['code', 'pseudo'], inplace=True)
train_df

,code_tokens,pseudo_tokens
0,"[l, =, mid, ;]","[set, l, to, mid]"
1,"[if, (, i, ==, 0, )]","[if, i, is, 0]"
2,"[cin, >>, n, >>, k, ;]","[read, n, and, k]"
3,"[long, long, min, =, LONG_LONG_MAX, ,, sum, =,...","[declare, long, longs, sum, =, 0, and, min, =,..."
4,"[int, dy, [, ], =, {, 0, ,, 0, ,, -, 1, ,, 1, ...","[dy, =, integer, array, where, the, the, follo..."
...,...,...
181857,"[static, const, int, mod, =, 1000000009, ;]","[declare, static, constant, integer, mod, =, 1..."
181858,"[cout, <<, "", NO, "", <<, ', \n, ', ;]","[print, NO, and, a, new, line]"
181859,"[ans, =, abs, (, x, -, y, ), /, d, ;]","[change, the, value, of, ans, to, abs, (, x, -..."
181860,"[else, if, (, s, <, f, )]","[else, if, s, is, less, than, f]"


In [95]:
X_train = pd.concat([train_df[['pseudo_tokens']], pred_df[['pred_code_gen_seq']], cpy_dataset[['pseudo_gen_seq']]], axis=1)
y_train = train_df[['code_tokens']]

In [96]:
X_train.head()

,pseudo_tokens,pred_code_gen_seq,pseudo_gen_seq
0,"[set, l, to, mid]","[[CPY], =, [CPY], ;]","[set, [CPY], to, [CPY]]"
1,"[if, i, is, 0]","[[CPY], (, [CPY], ==, [CPY], )]","[[CPY], [CPY], is, [CPY]]"
2,"[read, n, and, k]","[cin, >>, [CPY], >>, [CPY], ;]","[read, [CPY], and, [CPY]]"
3,"[declare, long, longs, sum, =, 0, and, min, =,...","[[CPY], [CPY], [CPY], [CPY], [CPY], ,, [CPY], ...","[declare, [CPY], longs, [CPY], [CPY], [CPY], a..."
4,"[dy, =, integer, array, where, the, the, follo...","[int, [CPY], [CPY], [CPY], [CPY], [CPY], [CPY]...","[[CPY], [CPY], integer, array, where, the, the..."


In [73]:
y_train.head()

,code_tokens
0,"[l, =, mid, ;]"
1,"[if, (, i, ==, 0, )]"
2,"[cin, >>, n, >>, k, ;]"
3,"[long, long, min, =, LONG_LONG_MAX, ,, sum, =,..."
4,"[int, dy, [, ], =, {, 0, ,, 0, ,, -, 1, ,, 1, ..."


In [74]:
input_words = pickle.load(open('input_words.pkl', 'rb'))
output_words = pickle.load(open('output_words.pkl', 'rb'))

pseudo_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
code_token_index = dict([(word, i+1) for i, word in enumerate(output_words)])

reverse_pseudo_index = dict((i, word) for word, i in pseudo_token_index.items())
reverse_code_index = dict((i, word) for word, i in code_token_index.items())

In [128]:
# Decision tree?
# print(pseudo_token_index['[CPY]'])
# print(pseudo_token_index['[UNK]'])

In [109]:
input_lexicon = defaultdict(lambda: pseudo_token_index[''], pseudo_token_index)
output_lexicon = defaultdict(lambda: code_token_index[''], code_token_index)

In [129]:
# input_lexicon['==']

In [135]:
importlib.reload(preprocessor)
X_train['features'] = X_train.apply(lambda row: preprocessor.create_combine_features(row, 'pred_code_gen_seq', 'pseudo_tokens', input_lexicon), axis=1)
X_train.head()

,pseudo_tokens,pred_code_gen_seq,pseudo_gen_seq,features
0,"[set, l, to, mid]","[[CPY], =, [CPY], ;]","[set, [CPY], to, [CPY]]","[{'word': 358, 'length': 5, 'is_numeric': Fals..."
1,"[if, i, is, 0]","[[CPY], (, [CPY], ==, [CPY], )]","[[CPY], [CPY], is, [CPY]]","[{'word': 358, 'length': 5, 'is_numeric': Fals..."
2,"[read, n, and, k]","[cin, >>, [CPY], >>, [CPY], ;]","[read, [CPY], and, [CPY]]","[{'word': 1981, 'length': 3, 'is_numeric': Fal..."
3,"[declare, long, longs, sum, =, 0, and, min, =,...","[[CPY], [CPY], [CPY], [CPY], [CPY], ,, [CPY], ...","[declare, [CPY], longs, [CPY], [CPY], [CPY], a...","[{'word': 358, 'length': 5, 'is_numeric': Fals..."
4,"[dy, =, integer, array, where, the, the, follo...","[int, [CPY], [CPY], [CPY], [CPY], [CPY], [CPY]...","[[CPY], [CPY], integer, array, where, the, the...","[{'word': 1057, 'length': 3, 'is_numeric': Fal..."


In [138]:
def copy_tags(row: pd.Series):
    """
    Combine the copy tags for a row
    """
    combination = []
    tokens_to_copy = []

    for i, token in enumerate(row['pseudo_gen_seq']):
        if token == '[CPY]':
            tokens_to_copy.append(row['pseudo_tokens'][i])

    token_iter =  iter(tokens_to_copy)

    for i, token in enumerate(row['pred_code_gen_seq']):
        if token == '[CPY]':
            try:
                combination.append(next(token_iter))
            except StopIteration:
                combination.append('')
        else:
            combination.append(token)
    
    return combination

In [144]:
y_train = pd.concat([train_df[['code_tokens']], X_train.apply(copy_tags, axis=1)], axis=1)
y_train.columns = ['code_tokens', 'code_gen_seq']

In [145]:
y_train

,code_tokens,code_gen_seq
0,"[l, =, mid, ;]","[l, =, mid, ;]"
1,"[if, (, i, ==, 0, )]","[if, (, i, ==, 0, )]"
2,"[cin, >>, n, >>, k, ;]","[cin, >>, n, >>, k, ;]"
3,"[long, long, min, =, LONG_LONG_MAX, ,, sum, =,...","[long, sum, =, 0, min, ,, =, LONG_LONG_MAX, , ;]"
4,"[int, dy, [, ], =, {, 0, ,, 0, ,, -, 1, ,, 1, ...","[int, dy, =, dy, [, 0, ], 0, ,, 0, ,, -, 1, ,,..."
...,...,...
181857,"[static, const, int, mod, =, 1000000009, ;]","[const, int, static, mod, =, ;]"
181858,"[cout, <<, "", NO, "", <<, ', \n, ', ;]","[cout, <<, NO, <<, endl, ;]"
181859,"[ans, =, abs, (, x, -, y, ), /, d, ;]","[ans, =, abs, (, x, -, y, ), /, d, ;]"
181860,"[else, if, (, s, <, f, )]","[else, if, (, s, <, f, )]"


## Strict accuracy

In [149]:
strict_bools = y_train.apply(lambda row: np.array_equal(row['code_tokens'], row['code_gen_seq']), axis=1)
strict_accuracy = strict_bools.sum() / len(y_train)

In [150]:
strict_accuracy

0.5531226974299194

In [131]:
# importlib.reload(preprocessor)
# y_train['code_indices'] = y_train.apply(lambda row: preprocessor.create_indices(row, 'code_tokens', output_lexicon), axis=1)
# y_train.head()

In [132]:

# from sklearn.tree import DecisionTreeClassifier
# from sklearn.feature_extraction import DictVectorizer
# from sklearn.pipeline import Pipeline

In [133]:
# X_train_total = [word for l in X_train['features'].tolist() for word in l]
# y_train_total = [word for l in y_train['code_indices'].tolist() for word in l]

In [134]:
# clf = Pipeline([
#     ('vectorizer', DictVectorizer()),
#     ('classifier', DecisionTreeClassifier(criterion='entropy'))
# ])

# clf.fit(X_train_total, y_train_total)   # Use only the first 10K samples if you're running it multiple times. It takes a fair bit :)
 
# print('Training completed')

### Seq2Seq

In [ ]:
from keras.models import 

In [ ]:
def seq2seq_model_builder():
    encoder_inputs = Input(shape=(MAX_LEN, ), dtype='int32',)
    encoder_embedding = embed_layer(encoder_inputs)
    encoder_LSTM = LSTM(HIDDEN_DIM, return_state=True)
    encoder_outputs, state_h, state_c = encoder_LSTM(encoder_embedding)
    
    decoder_inputs = Input(shape=(MAX_LEN, ), dtype='int32',)
    decoder_embedding = embed_layer(decoder_inputs)
    decoder_LSTM = LSTM(HIDDEN_DIM, return_state=True, return_sequences=True)
    decoder_outputs, _, _ = decoder_LSTM(decoder_embedding, initial_state=[state_h, state_c])
    
    # dense_layer = Dense(VOCAB_SIZE, activation='softmax')
    outputs = TimeDistributed(Dense(VOCAB_SIZE, activation='softmax'))(decoder_outputs)
    model = Model([encoder_inputs, decoder_inputs], outputs)
    
    return model